# Workbook 3: Informed Search Algorithms

## Introduction
This practical uses a simple easy-to-visualise optimisation problem to illustrate the properties of different search algorithms.
The problem is this:
- We are given  a model of a problem in terms of a graph  - which we can visualise as a maze.
- We are given a starting position and the desired endpoint (goal)
- The problem is to find a sequence of inputs that takes us from the start to the goal, preferably in as few moves as possible.

## Aims of this practical
1. To give you the opportunity to demonstrate your understanding by implementing the code needed to create different search algorithms.
2. To give you experience of comparing the behaviour of different search algorithms.
3. To give you experience of evaluating the efficiency of an algorithm for a problem ( in this case path-planning) by creating different instances of a problem (mazes) to *stress-test* different methods. 


<div class="alert alert-block alert-danger" style="color:black">
    <h2> This is an assessed practical</h2>
    <p> Each activity details how many marks it is worth (out of 100), and how marks are awarded.<br>
        As before, you will be submitting a file of python code for markingand  the notebooks use a two-stage process to help you.</p>
    <p>    As you work through the activities in the workbook:<ol>
            <li>  <em>Jupyter <code>%%writefile</code> cell magics</em>  save your code to a file"studentcode/student_wb3" as you run cells after editing</li>
            <li> Subsequent cells reload your code from that file to test the latest version .</li>
        </ol></p>
    When you are satisfied that you have completed the activities as required you should:</p>
    <ol>
        <li> Restart the kernel and do a single run through of every cell in turn  so <code>studentcode/student_wb3.py</code> has no duplicated definitions.</li>
        <li> Run the notebook <b>selfchecker.ipynb</b> to check for syntax errors when called from our marking code, and edit as needed</li>
            <li>Download <code>studentcode/student_wb3.py</code> to your local machine. </li>
        <li> Follow the links in the <i>Assessments</i> folder on Blackboard to submit it for automatic marking and feedback.<br>
            This may involve answering some multiple choices questions as well as submitting your code.</li>
        <li> <b>Read the feedback provided</b> and use it to improve your work.<br>
            <b>You will have four attempts to submit each notebook.</b></li>
    </ol>
<h3> Important instructions about formatting your code cells</h3>
<ol>
   <li>Where you are asked to insert code, or to complete a code skeleton, <b>you must only put code where indicated</b>. <br>
       Otherwise the marking server - or the plagiarism detection we will run later - will not be able to parse and accept them.<br>
       So your code should only go
       <ul> 
           <li><b>After</b> the lines marked <code># ====> insert your code below here </code></li>
           <li> <b>Before</b> the lines marked <code># <==== insert your code above here</code></li>
      </ul>
   </li>
               <li>The marking server will reject any files containing imports except "approvedimports".<br>
               The marking server has its own copy of that file, so there is no point editing the one we give you.</li>
<li>Even in comments you must not use any of these banned words: <em>system, read,import</li>
</ol>

</div>
 
   

## Background

As discussed in the lectures, we consider a general generate-and-test framwework for search that can be easily:
- adapted to provide algorithms with different behaviours
- applied to many different problems.
This is illustrated in the flowchart below.

<img src = "figures/generate-and-test-framework.png">

In this week's lecture presentation ( available in the notebook **W3_Inforned_Search_Algorithms.ipynb**)
we discussed how to create different algorithms, with different behaviours, just by adapting the function **select_and_move_from_openlist()**.

In last week's lab session you should have:
1. Got familiar with a general implementation framework we provided with classes for:
  - **SingleMemberSearch**
  - **Problem**
  - **CandidateSolution**
2. Created your own search subclasses **DepthfirstSearch** and **BreadthFirstSearch** 
  - by over-riding the function **select_and_move_from_openlist()**
3. Applied those algorithms to two sublasses of problem: **CombinationLock** and **FoxChickenGrain**


## This week
You will extend that work to:
1. Create your own implementations of the algorithms **LocalSearch**, **BestFirstSearch**, and **AStarSearch**
2. Test their behaviour on a simple maze problem 
   - because it is easy to visualise, and
   - many search graphs can be represented as  mazes.
3. **Test your understanding** by making mazes that *break* different algorithms.

<div style="color:black;background:#ECFFDC">
 <h2>Refresher: finding things in a list with python according to some criteria</h2>
    <h3> You can skip reading this box if you are comfortable with how to do this</h3>
    <ul>
        <li> A python list (let's call it <i>my_list</i>) holds a collection of objects, usually of the same type. </li>
        <li> If you have a list <i>my_list</i> with n elements, then <i>len(myList) = n</i> </li>
    <li> This is true whether the elements are chars, ints, float, or objects of some class</li>
    </ul>
    <p> If  <i>my_list</i> holds elements (objects) of a type that has an attribute <i>cost</i>, and we want to find the index (position) of the element one with the lowest value, we do it with a loop. <br>Start in position 0, then loop through every thing in the list one by one, looking at the value of <i>cost</i> in each element, remembering the position (index) of the one with the lowest <i>cost</i>.</p>
    <p> In code this looks like:</p>
    <pre lang="python" style="background:#ECFFDC">    
   best_index = 0
   <span style="color:green">for</span> i <span style="color:green">in range</span> (<span style="color:green">len</span> (my_list) ):    <span style="color:blue"># for historical reasons we often use i as the name of a loop variable </span>
       <span style="color:green">if</span> my_list[i].cost < my_list[best_index].cost : 
           best_index = i
   </pre>

<p> So at the end of this process the variable best_index tells us the index of the "best" element in that list according to our choice criteria (minimising cost).
<ul>
<li>If we want to use something else as our criteria, we just change the if statement.</li>
<li>Sometimes you might choose to store the value <i> best_so_far = my_list[best_index].cost</i> <br>
and use that in the comparison (line 3 above) to make your code more readable (shorter lines)<br>
- you just need to update <i>best_so_far</i> as well as <i>best_index</i> inside the <b>if</b> statement.</li>
</ul>
So this different version of the code does the same thing but you may find it easier to understand.
<pre lang="python" style="background:#ECFFDC">
    best_index = 0
    best_so_far = my_list[0].cost
    <span style="color:green">for</span> index <span style="color:green">in range</span> (len (my_list) ):
        this_cost= my_list[index].cost
        <span style="color:green">if</span> this_cost < best_so_far: 
            best_index = index
            best_so_far = this_cost
</pre>
</div>
                                       
                                       
                                       

# Part One: Familiarising yourself with the code framework
From last week you should be familiar with the basic classes in our framework:
- **Problem()**
- **CandidateSolution**
- **SingleMemberSearch**

and you should have created two subclasses:
- **DepthFirstSearch**
- **BreadthFirstSearch**

<p style="font-size:14px; color:black; font-weight:bold; font-style:italic;">
  If you have not done worksheet 2, go back and do that first.<br>
  <span style="color:red">Otherwise you will probably waste a lot of time.</span>
  <span style="color: black">and your tutors will just tell you to do that first.</span> 
</p>


If you want to look at the maze code it is in the file *maze.py*  
- it's a little complex, mostly to do with translating a lot between:
  - one-d arrays (cells have single index) and 
  - 2-d arrays (cells referenced by row  and column co-ordinates)
- so you may prefer to focus on your code implementation of different algorithms.

The file *maze.txt* provides a definition for **one specific** maze instance. 

<div class="alert alert-block alert-warning" style="color:black">
    <h1>Activity: Testing your Depth-first and Breadth-First Search code on the maze</h1>
    <h2>30 marks: 15 for each algorithm if it passes the test below on the marking server</h2>.
    Take the steps below to run and test your code from last week on the maze problems.<ol>
        <li> There are a series of code cells below import libraries and define useful functions. <br>
            A comment at the top of each cell will indicate whether you must run it, or it is optional.<br>
            The function <i> test_on_maze()</i> compares the behaviour of your implementation with mine - they should match.
       </li>
    <li> Where indicated copy-paste your class code for <b>DepthFirstSearch</b> from workbook2 into the empty cell below.<br>
            Then run that cell and the one after so you can see if your code solves the maze.<br>
             If there are errors fix them before you proceed.</li>
    <li> Where indicated copy-paste your class code for <b>BreadthFirstSearch</b> from workbook2 into the empty cell below.<br>
             Then run that cell and the one after so you can see if your code solves the maze.<br>
             If there are errors fix them before you proceed.</li>
    </ol>
    <p>If your code passed the tests for the <b>CombinationLock</b> problem last week it should work fine.<br>
        If not:</p><ul>
    <li>Edit your code in the cells below</li>
    <li> <b> Important</b> the __str__() method must return the same string as I use in <i>test_on_maze()</i> below</li>
    <li>Rerun those cells to tell python to use the edited versions.</li>
    <li> Then rerun the test cells as necessary</li>
    <li> You can turn on printing of the runlog in the test cells if it helps you debug your code.</li>
    </ul>
</div>

<div class="alert alert-block alert-warning" style="color:black">
    <h2> If you find the screen flickering disturbing</h2>
    <b> Apologies!</b>
       There is a trade-off between how long to pause after each move which affects the time taken to do a run, and how likely the screen is to flicker.  It can be hard to judge that with a distributed server.
    <ul>
        <li>edit the file <i>maze.py</i> in the jupyter editor. </li>
        <li> increase the default value of <i>refresh_rate</i> on line 107.</li>
        <li>rerun the first cell below to reload the Maze class</li></ul>
 </div>

**Run, but do not edit, the four cells below.**

In [ ]:
# YOU MUST RUN THIS CELL BUT DO NOT EDIT IT OR YOU WILL BREAK THE NOTEBOOK
import copy
import importlib
from importlib import reload
import sys, os
sys.path.append('../common')
sys.path.append('studentcode')

In [ ]:
# You MUST run this cell

from importlib import reload

# the libraries written for this course
from singlemembersearch import SingleMemberSearch
from candidatesolution import CandidateSolution

# doing it this way lets you edit maze.py then re-run this cell, without needing to restarting the kernel
import maze
reload(maze)
from maze import Maze

In [ ]:
# You MUST run this cell
# it defines the method that tries an algorithm on a maze

def run_on_maze(
    algorithm: SingleMemberSearch, 
    show_runlog: bool = False, 
    mazefile: str = "maze.txt"
     ) -> tuple[int, int]:
    """ function that tries to run a search algorithm on a maze problem
    Parameters
    ----------
    algorithm: name of a class of search algorithm
    show_runlog (bool) whether to print debugging information
    mazefile (str): name of the file containing  definition of a specific maze instance
    """
    
    mymaze = Maze(mazefile=mazefile)
    mysearch = algorithm(mymaze, constructive=True, max_attempts=1500)
    name = mysearch.__str__()
    trials = -1
    moves = -1
    found = mysearch.run_search()
    if found:
        trials = mysearch.trials
        moves = len(mysearch.result)
        print(
            f"search using {name} algorithm successful after {trials} attempts"
            f" length of path is {moves} moves."
        )
    else:
        print("solution not found in time allowed")
        if show_runlog:
            print(mysearch.runlog)

    del mymaze
    return trials, moves, name

In [ ]:
# You MUST run this cell

jims_results: dict = {
    "depth-first": [408, 77],
    "breadth-first": [1068, 57],
    "local search": [-1, -1],
    "best-first": [856, 57],
    "A Star": [812, 57],
}


def test_on_maze(algorithm: SingleMemberSearch, mazefile="maze.txt"):
    trials, moves, name = run_on_maze(algorithm, mazefile)
    correct_trials, correct_moves = jims_results[name]

    print(f"testing algorithm {name} on the simple maze.")
    if trials == -1 or moves == -1:
        errstr1 = "Error, the test suggests your code is not reaching the goal"
        errstr2 = "Error, the test suggests your code is not reaching the goal"

    else:
        errstr1 = (
            f"Error: your code is using {trials} trials "
            f" but should only need {correct_trials}.\n"
        )
        errstr2 = (
            f"Error: your code finds a solution with {moves} "
            f" but should only need {correct_moves}.\n"
        )
    assert trials == correct_trials, errstr1
    assert moves == correct_moves, errstr2
    print("test passed")

**Optionally** run the cell below to see an example of how help is automatically generated from doc strings in the code.

In [ ]:
# Optional
# Running this cell gives you a list of class methods and what they do
help(Maze)

### Copy-paste your class definition for Depth-first search into the code cell below then run it
- it must have no comments or code outside the class definition or the marking server will not accept it.

In [ ]:
%%writefile "studentcode/student_wb3.py"

from approvedimports import *

class DepthFirstSearch(SingleMemberSearch):
    """your implementation of depth first search to extend
    the superclass SingleMemberSearch search.
    Adds  a __str__method
    Over-rides the method select_and_move_from_openlist
    to implement the algorithm
    """
    # ====> insert your code below here


    # <==== insert your code above here

**Run the cell below to load your code from file and test it.**  
If you  get errors like ```ModuleNotFound``` make  sure you have run all the cells above

In [ ]:
# Run this cell to check your implementation works
#getting latest saved version of your code
from approvedimports import *

import student_wb3 
reload(student_wb3)


print("Testing Depth-First Search")
test_on_maze(student_wb3.DepthFirstSearch)

### Copy-paste your class definition for Breadth-first search into the code cell below then run it
- it must have no comments or code outside the class definition or the marking server will not accept it.

In [ ]:
%%writefile -a "studentcode/student_wb3.py"


class BreadthFirstSearch(SingleMemberSearch):
    """your implementation of depth first search to extend
    the superclass SingleMemberSearch search.
    Adds  a __str__method
    Over-rides the method select_and_move_from_openlist
    to implement the algorithm
    """
    # ====> insert your code below here

    # <==== insert your code above here

In [ ]:
# Run this cell to test your breath-first implementation
#getting latest saved version of your code
reload(student_wb3)


print("Testing Breadth-First Search")
test_on_maze(student_wb3.BreadthFirstSearch)

<div class = "alert alert-warning" style="color:black">
    <h1> Activity Two: Implementing Best-First and A Star Search</h1>
    <h2> 40 Marks (20 each for an implementation that passes the tests)</h2>
    <p> For these two algorithms the cells below provide the pseudo-code, a partially completed implementation, and a test.</p> <p>This activity requires you to complete the code to produce classes which implement the algorithms, using the pseudocode as your guide.</p>
        <p> You are strongly advised to proceed by:</p> <ol>
        <li> Copying the pseudo-code into an appropriate place in the class code</li>
        <li> Turning the pseudo-code into comments with spaces between for your code</li> 
        <li> Your code should start by checking whether the openlist is empty <ul>
            <li>  return None if openlist is empty</li>
            <li> Otherwise select and return the appropriate item from the openlist </li>
            </ul>
    <li> Then answer the multiple choice questions to check your understanding</li>
        </ol>
        <h3>Hints:</h3><ul>
               <li> Use the reminder at the start of this workbook for how to select from a list by value</li>
<li> Remember that each instance of the  class defines an attribute <i> self.open_list</i> </li>
       <li>  Everything on that list should be of type <b>CandidateSolution</b> <br>
           and have attributes <i> quality</i> and <i>variable_values</i>.</li>
        <li> For any list <i>mylist</i> you can query how many things it holds via <i>len(mylist)</i></li>
        </ul>
    </div>

## Pseudocode for function SelectAndMoveFromOpenList in Best-First Search

<div style="background:#F0FFFF">
<p style="color:darkredmargin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd>bestChild &larr; <b>GetMemberWithHighestQuality</b>(openList)</dd>
    <dd> RETURN bestChild&nbsp;&nbsp;&nbsp;&nbsp;<span style="background:pink">Best-First keeps the openlist to allow backtracking</span></dd>
</dl>
</div>   

In [ ]:
%%writefile -a "studentcode/student_wb3.py"

class BestFirstSearch(SingleMemberSearch):
    """Implementation of Best-First   search.
    You need to complete this
    """
    # ====> insert your code below here

    # <==== insert your code above here

In [ ]:
# run this to test your implementation
#getting latest saved version of your code
reload(student_wb3)

print("Testing Best-First Search")
test_on_maze(student_wb3.BestFirstSearch)

## Pseudocode for function SelectAndMoveFromOpenList in AStar Search

<div style="background:#F0FFFF">
<p style="color:darkred;margin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd><span style="background:pink">AStar picks using sum of quality +cost</span></dd>
    <dd>bestChild &larr; <b>GetMemberWithHighestCombinedScore</b>(openList)</dd>
    <dd> RETURN bestChild&nbsp;&nbsp;&nbsp;&nbsp;</dd>
</dl>
</div>   
<div style="background:white"> <h3>Note that</h3><ul>
    <li>This is just like best-first with a modified selection.</li>
    <li> To make more efficient you can track <i>bestSoFar</i> and modify <b>UpdateWorkingMemory()</b><br>
        so it doesn't put things on the openlist if depth > bestSoFar </li></ul>  </div> 

In [ ]:
%%writefile -a "studentcode/student_wb3.py"

class AStarSearch(SingleMemberSearch):
    """Implementation of A Star  search.
    You need to complete this
    """
    # ====> insert your code below here

    # <==== insert your code above here

In [ ]:
# run this to test your A Star algorithm

#getting latest saved version of your code
reload(student_wb3)

test_on_maze(student_wb3.AStarSearch)

<div class="alert alert-warning" style="color:black">
    <h1> Activity 3: Testing your understanding</h1>
    <h2> 20 marks</h2>
    Run the cell below and answer the questions then press the <b>check</b> button to check your answers. <br>
    When you submit your jupyter notebook for automated marking via blackboard, <br>
    there will be similar questions for you to answer online.
    </div>
    

     

In [ ]:
import workbook3_utils as wb3

reload(wb3)
display(wb3.Q1)
display(wb3.Q2)
display(wb3.Q3)
display(wb3.Q4)
display(wb3.Q5)
display(wb3.Q6)
display(wb3.Q7)
display(wb3.Q8)

<div class="alert alert-warning" style="color:black">
    <h2> Activity 4: Testing your understanding by creating new instances to <it>break</it> algorithms</h2>
    <h3> 10 Marks (5 for each new maze)</h3>
    <p> The first two cells below illustrate how to create a new instances of the path-finding problem by changing walls into paths or vice-versa.</p>
    <p> The third cell  shows how to save an edited maze to file and then checks it loads properly into a new maze object</p>
    <p><b> Task: Experiment with creating new mazes then:</b></p>
    <ol>
        <li> Create a maze in which depth-first search gets stuck in a loop<br>
            and save your new maze to file called <i>studentcode/maze-loop.txt</i> </li>
        <li> Create a maze in which breadth-first search finds the goals in fewer trials than depth-first<br>
            and save this to file <i>studentcode/maze-odd.txt</i></li>
        <li> On the marking server you will be asked to upload these two files.</li>
    </ol>
    </div>

**Run (but do not edit) the three cells below** to see how to edit and save  mazes.

In [ ]:
# this is an example with 'before and after' displays of how to make a hole in a wall
# in this case one place to the right of the entrance, three blocks down
# the first value is for the row of the cell to change, the second for the column
wall_colour= 0.0
hole_colour = 1.0
hole_in_wall = Maze(mazefile="maze.txt")
hole_in_wall.contents[2][10] = hole_colour
hole_in_wall.show_maze()

In [ ]:
# demonstration of placing a new wall

# load maze from originnal file
new_wall = Maze(mazefile="maze.txt")

# edit it to place a new wall- 2 rows down and three columns to the right of the entramce
new_wall.contents[2][13] = wall_colour
new_wall.show_maze()


In [ ]:
# saving and reloading mazes 

# save edited maze to new file
new_wall.save_to_txt("maze-newwall.txt")


# reload into new maze object
print('this is the reloaded maze')
reloaded_maze = Maze(mazefile="maze-newwall.txt")

# test they have the same contents
assert reloaded_maze.contents == new_wall.contents
print("the reloaded maze matches the version still in memory")

### Below are two sets of cells which let you design and test the two use cases.
You need to **edit and run** the first cell of each pair.
- combine and adapt the code snippets above to:
  - load the original maze
  - edit it for the two use cases
  - save each to a file with the name specified

**Then run the second cell of each pair** to test the mazes you have produced.

In [ ]:
%%writefile -a "studentcode/student_wb3.py"
wall_colour= 0.0
hole_colour = 1.0

def create_maze_breaks_depthfirst():
    pass
    # ====> insert your code below here

    # <==== insert your code above here


In [ ]:
def test_maze_that_breaks_depthfirst():
    
    outstr= ""
    #depth first should not complete the maze
    depth_behaviour = run_on_maze(student_wb3.DepthFirstSearch, show_runlog=False,mazefile='studentcode/maze-breaks-depth.txt') 
    assert depth_behaviour[1] == -1,' depth first should not finish on the maze'
    assert depth_behaviour[0] == -1,' depth first should be terminated after 1500 trials'
    outstr +=' your maze defeates depth-first search '
    
    #breadth first should
    breadth_behaviour = run_on_maze(student_wb3.BreadthFirstSearch, show_runlog=False,mazefile='studentcode/maze-breaks-depth.txt') 
    assert breadth_behaviour[0] >0, ' breadth first should solve maze after enough trials'
    assert breadth_behaviour[1] >0, ' breadth first should solve maze with path >0 moves'
    outstr+= f'breadth-first can solve your maze in {breadth_behaviour[0]} trials'
    outstr +='test passed'
    print(outstr)

#getting latest saved version of your code
reload(student_wb3)
student_wb3.create_maze_breaks_depthfirst()
test_maze_that_breaks_depthfirst()

In [ ]:
%%writefile -a "studentcode/student_wb3.py"
def create_maze_depth_better():
    pass
    # ====> insert your code below here 

    # <==== insert your code above here


    

In [ ]:
def test_maze_depth_better():
    depth_behaviour = run_on_maze(student_wb3.DepthFirstSearch, show_runlog=False,mazefile='studentcode/maze-depth-better.txt')
    breadth_behaviour = run_on_maze(student_wb3.BreadthFirstSearch, show_runlog=False,mazefile='studentcode/maze-depth-better.txt') 
    
    #should both find the goal state
    #but I can't know how long the path is on your maze
    assert depth_behaviour[1] != -1, 'error: depth first is not finding the goal state'
    assert breadth_behaviour[1] != -1, 'error: breadth first is not finding the goal state'
    assert depth_behaviour[1] <= breadth_behaviour[1], 'error: depth first should not find a longer path'
    print('both find goal state and depth-first path is as good or shorter')
    
    # depth first should use fewer trials
    assert depth_behaviour[0] != -1, 'error: depth first is not finding the goal state'
    assert breadth_behaviour[0]!= -1, 'error: brteadth first is not finding the goal state'
    assert  depth_behaviour[0] < breadth_behaviour[0], 'error: depth first should take fewer trials'
    print(' depth- first needs fewer attempts')
    print('test passed - you designed your maze well')

#getting latest saved version of your code
reload(student_wb3)
student_wb3.create_maze_depth_better()
test_maze_depth_better()

<div class="alert alert-block alert-success" style="color:black"><b>Save and close Jupyter:</b>
    <ol>
        <li>Use the jupyterlab functions to download your work (ask your tutor if you need help with this) and save it somewhere sensible so you can find it easily.</li>
        <li>Shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook)</li>
    </ol>
</div>